In [2]:
from pandas import read_csv
import sys
sys.path.append('..')  # Add the parent directory to the Python path

# Now you can import modules from the parent directory
from src.utils import constants

class RawDataReader():
    def __init__(self) -> None:
        constants.DATA_DIR
        pass

In [ ]:
class RawData